 #  <CENTER>Sistema de recomendacion para fichaje de jugadores

Para el presente proyecto se ha realizado la siguiente metodología:

## Primera parte:

 ###  <CENTER>Scraping SOFIFA

<img src="https://yt3.ggpht.com/a/AGF-l79MYDjAxxN_0vCGTf1tJYk0JaZYsdo8-Fmz3A=s900-mo-c-c0xffffffff-rj-k-no" width="300"> <center> 

In [1]:
#Librerias scraping
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import pandas as pd

In [2]:
# url de la pagina principal
url_generica = 'http://sofifa.com/players?offset=0'


In [3]:
#Funcion para obtener soup
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

#### Obtener links de jugadores en una página

In [4]:
#Funcion para obtener los links de los jugadores de una pagina
def find_allpage_players(soup):
    final_details = {}
    table = soup.find('table', {'class': 'table-hover'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a', {'class': 'nowrap'})
    return(['http://sofifa.com' + player['href']+'?units=mks&currency=USD' for player in all_a])

#### Obtener datos del top de la cartilla del jugador

In [5]:
def player_top_info(soup):
    player_data = {}
    #link de la imagen
    player_data['pic'] = soup.find('img')['data-src']
    #Nombre del jugador
    player_data['nombre_init'] = soup.find('h1').text.split(' (')[0]
    #Valor del jugador
    player_data['value']= soup.find('span', attrs={'class': None}).text

    #observar estructura del soup 
    #print (spannn.prettify())

    # String con datos de nombre + pais + edad + fecha_nac + altura + peso
    spannn =soup.find('div', {'class': 'meta bp3-text-overflow-ellipsis'})

    #Nombre completo del jugador
    player_data['full_name']=spannn.contents[0]

    #Posicion del jugador
    Posicion=spannn.find_all('span')
    lista_posicion=[]
    for x in Posicion:
        lista_posicion.append(x.text)
    player_data['best_position']=','.join(lista_posicion)

    #País del jugador
    player_data['birth_country']=spannn.find('a')['title']

    #Observando con la funcion prettify se puede obtener el ultimo valor contenido 
    #el cual posee los siguientes valores: 
    #Edad, Fec_nac, Medida, Peso
    #---------------------------------
    last_content=spannn.contents[len(spannn.contents)-1] #EJEM: 'Age 22 (Apr 3, 1996) 6\'2" 154lbs'

    #Edad jugador
    player_data['Age']=last_content.split('(')[0].split()[1]

    #parenthesis remove process
    remove=re.split(r'[()]',last_content)  # [' Age 22 ', 'Apr 3, 1996', ' 6\'2" 154lbs']

    #Fecha de nacimiento del jugador
    player_data['Fec_nac']=remove[1] 

    #Altura del jugador
    len_lcont=len(last_content.split())
    player_data['altura']=last_content.split()[len_lcont-2]

    #Peso del jugador
    player_data['peso']=last_content.split()[len_lcont-1]

    #print (spannn.prettify())
    return (player_data)

#### Obtener statistics del top de la cartilla del jugador

In [6]:
def find_player_stats(soup):
    player_stats = {}
    infox=soup.find_all('div',{"class":"column col-4 text-center"})
    #Rating global de jugador 
    rating=infox[0].text.split(' ')[0]
    if rating.find('+')>=0: 
        player_stats['rating'] = int(rating.split('+')[0])+int(rating.split('+')[1])
    elif rating.find('-')>=0:
        player_stats['rating'] = int(rating.split('-')[0])-int(rating.split('-')[1])
    else:
        player_stats['rating'] = int(rating)
    #Potencial del jugador
    potential=infox[1].text.split(' ')[0]
    if potential.find('+')!=-1: 
        player_stats['potential'] = int(potential.split('+')[0])+int(potential.split('+')[1])
    elif potential.find('-')>=0:
        player_stats['potential'] = int(potential.split('-')[0])-int(potential.split('-')[1])
    else:
        player_stats['potential'] = int(potential)
    #Salario semanal del jugador
    player_stats['wage'] = infox[3].find_all('span')[0].text
    return (player_stats)

#### Obtener datos secundarios de la cartilla de jugador

In [7]:
def find_player_secondary_info(soup):
    #secondary info
    player_data={}
    #Pie preferido
    player_data['preff_foot'] = soup.find('label', text='Preferred Foot').parent.contents[2].strip('\n')
    #Club
    player_data['club_name'] = soup.find_all('ul')[1].find('a').text
    #Posicion del club
    player_data['club_pos'] = soup.find('label', text='Position').parent.find('span').text
    #Numero de camiseta del jugador en el club
    player_data['club_jersey'] = soup.find('label', text='Jersey Number').parent.contents[1].strip('\n')
    soup.find('label', text='Jersey Number').parent.contents[1].strip('\n')
    #Costo de clausula de liberación
    if soup.find('label',text='Release Clause'):
        player_data['release_clause']=soup.find('label',text='Release Clause').parent.find('span').text
    else:
        player_data['release_clause']=''
    #Datos del jugador en su club
    if soup.find('label', text='Joined'):
        player_data['club_joined'] = soup.find('label', text='Joined').parent.contents[2].strip('\n')
        player_data['contract_valid'] = soup.find('label', text='Contract Valid Until').parent.contents[2].strip('\n')
    else:
        player_data['club_joined'] = ''
        player_data['contract_valid'] =''
    #Datos del jugador como seleccion 
    if len(soup.find_all('ul')) > 2:
        player_data['country'] = soup.find_all('ul')[2].find('a').text
        player_data['c_position'] =soup.find_all('ul')[2].find('label',text='Position').parent.find('span').text
        player_data['country_num_jersey'] =soup.find_all('ul')[2].find('label',text='Jersey Number').parent.text.strip('Jersey Number')
    else:
        player_data['country'] = ''
        player_data['c_position'] =''
        player_data['country_num_jersey']=''
    #especialidad del jugador
    if soup.find('div',{'class':'mt-2'}).txt!=None:
        player_data['speciality'] =soup.find('div',{'class':'mt-2'}).text.split()[0].strip('#') #Hasta ahora solo extrae uno
    else:
        player_data['speciality'] =''

    return (player_data)

#### Obtener atributos del jugador (parte bottom)

In [8]:
fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy',                            #FK Accuracy
 'Short Passing', 'Volleys', 'Dribbling', 'Curve',
 'FK Accuracy', 'Long Passing', 'Ball Control',
 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
 'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
 'Long Shots', 'Aggression', 'Interceptions', 'Positioning',
 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle',
 'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking',
 'GK Positioning', 'GK Reflexes']

In [9]:
def find_fifa_priminfo(soup):
    player_data = {}

    bl_1 =soup[6].find_all('li')

    bl_lis =soup[7].find_all('li')+bl_1

    #player stats [Attacking, Skill ... Goalkepper]
    for li in bl_lis:
        for stats in fifa_stats:
            if stats in li.text:
                value=li.text.split(' ')[0].replace('\n', '')
                if value.find('+')>=0: 
                    a=value.split('+')
                    player_data[stats.replace(' ', '_').lower()] =int(a[0])+int(a[1])
                elif value.find('-')>=0: 
                    a=value.split('-')
                    player_data[stats.replace(' ', '_').lower()] =int(a[0])-int(a[1])                
                else:
                    player_data[stats.replace(' ', '_').lower()] = int((li.text.split(' ')[0]).replace('\n', ''))


    #FALTA MEJORAR
    #traits 
    try:
        traits=fifa_info[7].find_all('h5',text='Traits')[0]
    except:
        traits=None
        #print("An exception occurred")

    if traits:
        player_data['traits'] =traits.parent.text.split('\n')[1]
    else:
        player_data['traits']=''

    return(player_data)

#### Función para obtener todos los datos de un jugador

In [10]:
def player_all_details(url):
    all_details={}
    soup = soup_maker(url)
    #General- Cartilla total del jugafor (contenedor de todos los datos)
    player_info = soup.find('div', {'class': 'bp3-card player'})
    all_details.update(player_top_info(player_info))
    #Estadisticas del jugador
    player_stats = soup.find('div', {'class': 'stats'})
    all_details.update(find_player_stats(player_stats))
    #Datos del club (1 seccion)
    secondary_info = soup.find('div', {'class': 'teams spacing'})
    all_details.update(find_player_secondary_info(secondary_info))
    #Datos de la Fifa (Attacking - Mentality etc - son 6 secciones)
    fifa_info = soup.find_all('div', {'class': 'spacing'})
    all_details.update(find_fifa_priminfo(fifa_info))
    return(all_details)

#### Funcion para obtener link de siguiente pagina

In [11]:
#Funcion para obtener los link de las siguiente pagina dada una pagina actual de sofifa
def find_next_page_to_search(url_actual):
    soup_try = soup_maker(url_actual)
    x=soup_try.find('div',{'class':'pagination'}).find_all('a')
    page=x[len(x)-1]['href']
    return 'https://sofifa.com/players?type=all&lg%5B0%5D=13&'+page.split('?')[1]

    #https://sofifa.com/players?type=all&lg%5B0%5D=13&offset=60
    #find_next_page_to_search('https://sofifa.com/players?type=all&lg%5B0%5D=13&offset=60')

In [12]:
def find_all_pages(url_base):
    list_url=['',url_base]
    for x in range (1,20):
        time.sleep(3)
        next_url=find_next_page_to_search(list_url[x])
        if list_url[x-1]==next_url:
            return list_url
        elif list_url[x]!=next_url:
            list_url.append(find_next_page_to_search(list_url[x]))
    return print('fail')

In [13]:
def DB_League(initial_url_league):
    #obtener soup de una pagina con 60 jugadores 
    soup=soup_maker(initial_url_league)

    #Generar base de datos
    database_league=pd.DataFrame()
    #obtener lista de links de todos los jugadores de esa página
    links_play=find_allpage_players(soup)
    #obtener un valor random de un jugador
    p=player_all_details(links_play[5])
    #asignar cabeceras de acuerdo al diccionario armado
    database_league = database_league.assign(**p) 

    #Obtener lista total de paginas a escrapear
    lista_url=find_all_pages(initial_url_league)
    lista_url.remove("") 

    #Armar base de datos

    for link_page in lista_url:
        print(link_page)
        #obtener lista de links de todos los jugadores de esa página
        links_depor=find_allpage_players(soup_maker(link_page))
        #recorrer cada jugador de la página
        for x in range(0,len(links_depor)):
            time.sleep(3)
            data=player_all_details(links_depor[x])
            database_league = database_league.append(data, ignore_index=True)
    return database_league

 #  <center> Armar BDs

### <center><i>Data Premier League - ENGLAND
  <img src="https://techcentral.co.za/wp-content/uploads/2018/01/premier-league-2156-1120.jpg" width="600"> <center> 


In [14]:
#URL English Premier League
url='https://sofifa.com/players?type=all&lg%5B0%5D=13&offset=0'
#URL English Premier League $
url_premier_league='https://sofifa.com/players?type=all&lg%5B0%5D=13&currency=USD&offset=0'

In [15]:
#now=time.time()
#premier_league=DB_League(url_premier_league)
#print(time.time()-now)

In [16]:
#premier_league

In [17]:
#GUARDAR DB
#premier_league.to_csv('Proyecto/premier_league_SOFIFA_USD.csv',index=False,encoding='UTF-16')

### <center><i>Data La Liga - SPAIN
  <img src="https://imagekit.androidphoria.com/wp-content/uploads/Ver-partidos-de-la-liga-espanola-en-directo-en-Android.jpg" width="600"> <center> 


In [18]:
url_laliga='https://sofifa.com/players?type=all&lg%5B0%5D=53&offset=0'

In [19]:
#la_liga=DB_League(url_laliga)
#la_liga

In [ ]:
#GUARDAR DB
#premier_league.to_csv('Proyecto/premier_league_SOFIFA_USD.csv',index=False,encoding='UTF-16')

### <center><i>Data Bundesliga - GERMANY
  <img src="http://www.comexmasters.com/wp-content/uploads/2018/08/Bundesliga-portada.jpg" width="600"> <center> 


In [21]:
url_bundesliga='https://sofifa.com/players?type=all&lg%5B0%5D=19&offset=0'

In [22]:
bundesliga=DB_League(url_bundesliga)
bundesliga

https://sofifa.com/players?type=all&lg%5B0%5D=19&offset=0
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=60
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=120
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=180
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=240
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=300
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=360
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=420
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=480
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=19&offset=540


,pic,nombre_init,value,full_name,best_position,birth_country,Age,Fec_nac,altura,peso,...,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,traits
0,https://cdn.sofifa.org/players/10/19/188545.png,R. Lewandowski,$89.3M,Robert Lewandowski,ST,Poland,29,"Aug 21, 1988",184cm,80kg,...,78,78,90,79,88,84,78,84,84,
1,https://cdn.sofifa.org/players/10/19/167495.png,M. Neuer,$39.4M,Manuel Neuer,GK,Germany,32,"Mar 27, 1986",193cm,92kg,...,60,51,83,35,25,77,43,80,16,
2,https://cdn.sofifa.org/players/10/19/178603.png,M. Hummels,$53.4M,Mats Hummels,CB,Germany,29,"Dec 16, 1988",191cm,92kg,...,56,58,87,58,71,68,66,84,51,
3,https://cdn.sofifa.org/players/10/19/188350.png,M. Reus,$67.3M,Marco Reus,"CAM,LM,ST",Germany,29,"May 31, 1989",180cm,71kg,...,85,86,88,82,88,70,76,67,86,
4,https://cdn.sofifa.org/players/10/19/189509.png,Thiago,$59.7M,Thiago Alcântara do Nascimento,"CM,CAM,CDM",Spain,27,"Apr 11, 1991",174cm,70kg,...,69,90,86,86,76,75,75,59,79,
5,https://cdn.sofifa.org/players/10/19/198710.png,J. Rodríguez,$68.4M,James David Rodríguez Rubio,"CAM,CM,RM",Colombia,26,"Jul 12, 1991",180cm,75kg,...,66,82,85,76,86,54,70,68,92,
6,https://cdn.sofifa.org/players/10/19/212622.png,J. Kimmich,$52.2M,Joshua Kimmich,"RB,CM,CDM",Germany,23,"Feb 8, 1995",176cm,73kg,...,68,84,86,83,75,79,94,64,69,
7,https://cdn.sofifa.org/players/10/19/189596.png,T. Müller,$52.2M,Thomas Müller,"CAM,RM,RW,CF",Germany,28,"Sep 13, 1989",186cm,75kg,...,75,74,91,72,78,81,87,68,80,
8,https://cdn.sofifa.org/players/10/19/189117.png,R. Bürki,$34.8M,Roman Bürki,GK,Switzerland,27,"Nov 14, 1990",187cm,85kg,...,54,49,83,52,22,74,36,74,14,
9,https://cdn.sofifa.org/players/10/19/177413.png,A. Witsel,$34.8M,Axel Witsel,"CDM,CM",Belgium,29,"Jan 12, 1989",188cm,81kg,...,73,77,85,67,78,76,86,82,77,


In [23]:
#GUARDAR DB
bundesliga.to_csv('Proyecto/bundesliga_SOFIFA_USD.csv',index=False,encoding='UTF-16')

### <center><i>Data Ligue 1 - FRANCE
  <img src="https://www.ngolos.com/assets/images/thumbnails/France/Ligue-1.jpg" width="600"> <center> 





In [24]:
url_ligue1='https://sofifa.com/players?type=all&lg%5B0%5D=16&offset=0'
ligue1=DB_League(url_ligue1)

https://sofifa.com/players?type=all&lg%5B0%5D=16&offset=0
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=60
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=120
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=180
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=240
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=300
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=360
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=420
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=480
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=540
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=16&offset=600


In [25]:
ligue1

,pic,nombre_init,value,full_name,best_position,birth_country,Age,Fec_nac,altura,peso,...,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,traits
0,https://cdn.sofifa.org/players/10/19/190871.png,Neymar Jr,$125.3M,Neymar da Silva Santos Junior,"LW,CAM",Brazil,26,"Feb 5, 1992",175cm,68kg,...,90,96,94,84,80,61,81,49,82,
1,https://cdn.sofifa.org/players/10/19/179813.png,E. Cavani,$69.6M,Edinson Roberto Cavani Gómez,ST,Uruguay,31,"Feb 14, 1987",185cm,77kg,...,76,77,91,59,87,88,92,78,79,
2,https://cdn.sofifa.org/players/10/19/164240.png,Thiago Silva,$27.8M,Thiago Emiliano da Silva,CB,Brazil,33,"Sep 22, 1984",183cm,82kg,...,69,68,82,68,71,90,74,82,68,
3,https://cdn.sofifa.org/players/10/19/231747.png,K. Mbappé,$95.1M,Kylian Mbappé,"ST,RW",France,19,"Dec 20, 1998",178cm,73kg,...,96,92,87,83,80,75,83,74,79,
4,https://cdn.sofifa.org/players/10/19/199556.png,M. Verratti,$63.8M,Marco Verratti,"CM,CDM",Italy,25,"Nov 5, 1992",165cm,60kg,...,62,90,85,94,56,64,77,52,58,
5,https://cdn.sofifa.org/players/10/19/1179.png,G. Buffon,$3.5M,Gianluigi Buffon,GK,Italy,40,"Jan 28, 1978",192cm,92kg,...,38,55,82,49,39,75,39,69,13,
6,https://cdn.sofifa.org/players/10/19/199482.png,A. Lopes,$36M,Anthony Lopes,GK,Portugal,27,"Oct 1, 1990",184cm,81kg,...,64,67,84,62,20,82,36,73,18,
7,https://cdn.sofifa.org/players/10/19/207865.png,Marquinhos,$50.5M,Marcos Aoás Corrêa,"CB,CDM",Brazil,24,"May 14, 1994",183cm,75kg,...,76,69,85,68,61,88,75,75,35,
8,https://cdn.sofifa.org/players/10/19/183898.png,A. Di María,$42.3M,Ángel Fabián Di María Hernández,"RW,LW",Argentina,30,"Feb 14, 1988",180cm,69kg,...,84,92,82,77,83,72,78,60,80,
9,https://cdn.sofifa.org/players/10/19/216594.png,N. Fekir,$47M,Nabil Fekir,CAM,France,24,"Jul 18, 1993",173cm,75kg,...,79,84,80,91,84,64,77,82,82,


In [26]:
#GUARDAR DB
ligue1.to_csv('Proyecto/ligue1_SOFIFA_USD.csv',index=False,encoding='UTF-16')

### <center><i>Data Scottish - SCOTLAND
  <img src="https://ichef.bbci.co.uk/onesport/cps/624/cpsprodpb/105C4/production/_104021076_spfl_premiership_logo_sns.jpg" width="600"> <center> 

In [27]:
url_scottish='https://sofifa.com/players?type=all&lg%5B0%5D=50&offset=0'
scottish=DB_League(url_scottish)

https://sofifa.com/players?type=all&lg%5B0%5D=50&offset=0
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=50&offset=60
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=50&offset=120
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=50&offset=180
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=50&offset=240
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=50&offset=300


In [28]:
scottish

,pic,nombre_init,value,full_name,best_position,birth_country,Age,Fec_nac,altura,peso,...,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,traits
0,https://cdn.sofifa.org/players/10/19/200630.png,J. Forrest,$11M,James Forrest,"RM,RWB",Scotland,26,"Jul 7, 1991",175cm,77kg,...,92,92,72,90,70,74,77,64,70,
1,https://cdn.sofifa.org/players/10/19/226491.png,K. Tierney,$12.8M,Kieran Tierney,"LB,LWB,LM",Scotland,21,"Jun 5, 1997",178cm,78kg,...,91,76,72,77,79,69,88,76,66,
2,https://cdn.sofifa.org/players/10/19/134820.png,S. Brown,$3.6M,Scott Brown,CDM,Scotland,33,"Jun 25, 1985",175cm,80kg,...,66,74,72,73,70,77,85,77,48,
3,https://cdn.sofifa.org/players/10/19/244269.png,F. Benković,$12.2M,Filip Benković,CB,Croatia,20,"Jul 13, 1997",194cm,91kg,...,66,52,70,44,74,76,75,86,44,
4,https://cdn.sofifa.org/players/10/19/207928.png,T. Rogić,$11M,Tom Rogić,"CAM,CM",Australia,25,"Dec 16, 1992",189cm,88kg,...,66,74,64,75,82,52,58,77,80,
5,https://cdn.sofifa.org/players/10/19/50542.png,J. Defoe,$4.1M,Jermain Defoe,ST,England,35,"Oct 7, 1982",171cm,65kg,...,63,77,77,87,81,68,55,54,78,
6,https://cdn.sofifa.org/players/10/19/211093.png,C. McGregor,$8.1M,Callum McGregor,"CDM,CAM,LM,LB",Scotland,25,"Jun 14, 1993",174cm,74kg,...,78,86,72,84,67,53,90,68,72,
7,https://cdn.sofifa.org/players/10/19/169214.png,S. Sinclair,$7.5M,Scott Sinclair,LM,England,29,"Mar 25, 1989",178cm,70kg,...,91,90,74,82,76,66,85,62,66,
8,https://cdn.sofifa.org/players/10/19/197837.png,D. Boyata,$7.5M,Anga Dedryck Boyata,CB,Belgium,27,"Nov 28, 1990",188cm,86kg,...,72,62,55,59,56,85,77,90,25,
9,https://cdn.sofifa.org/players/10/19/49000.png,A. McGregor,$1M,Allan McGregor,GK,Scotland,36,"Jan 31, 1982",183cm,74kg,...,41,65,70,51,23,73,30,71,14,


In [29]:
#GUARDAR DB
scottish.to_csv('Proyecto/scottish_SOFIFA_USD.csv',index=False,encoding='UTF-16')

### <center><i>Data Football league - ENGLAND
  <img src="https://upload.wikimedia.org/wikipedia/de/thumb/8/8b/The_Football_League_Logo.svg/1200px-The_Football_League_Logo.svg.png" width="600"> <center> 


In [32]:
url_2da_England='https://sofifa.com/players?type=all&lg%5B0%5D=14&offset=0'
da2_England=DB_League(url_2da_England)

https://sofifa.com/players?type=all&lg%5B0%5D=14&offset=0
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=60
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=120
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=180
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=240
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=300
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=360
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=420
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=480
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=540
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=600
https://sofifa.com/players?type=all&lg%5B0%5D=13&type=all&lg%5B0%5D=14&offset=660


In [33]:
da2_England

,pic,nombre_init,value,full_name,best_position,birth_country,Age,Fec_nac,altura,peso,...,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,traits
0,https://cdn.sofifa.org/players/10/19/203042.png,J. Butland,$16.2M,Jack Butland,GK,England,25,"Mar 10, 1993",196cm,95kg,...,55,55,73,50,29,76,40,79,13,
1,https://cdn.sofifa.org/players/10/19/177644.png,Kiko Casilla,$8.7M,Francisco Casilla Cortés,GK,Spain,31,"Oct 2, 1986",190cm,86kg,...,49,43,74,58,24,69,43,79,13,
2,https://cdn.sofifa.org/players/10/19/194201.png,P. Jansson,$9.9M,Pontus Jansson,CB,Sweden,27,"Feb 13, 1991",196cm,86kg,...,69,53,69,31,68,54,71,90,50,
3,https://cdn.sofifa.org/players/10/19/206517.png,J. Grealish,$13.3M,Jack Grealish,"CAM,CM",England,22,"Sep 10, 1995",175cm,68kg,...,74,85,68,82,67,34,72,55,72,
4,https://cdn.sofifa.org/players/10/19/179591.png,Pablo Hernández,$5.4M,Pablo Hernández Domínguez,"RM,CM,LM",Spain,33,"Apr 11, 1985",173cm,66kg,...,69,79,68,84,74,67,78,53,74,
5,https://cdn.sofifa.org/players/10/19/210648.png,A. Hegazi,$8.7M,Ahmed Hegazi,CB,Egypt,27,"Jan 25, 1991",194cm,95kg,...,55,45,65,51,27,75,73,85,15,
6,https://cdn.sofifa.org/players/10/19/169792.png,J. Rodriguez,$9.3M,Jay Rodriguez,"ST,LW",England,28,"Jul 29, 1989",185cm,84kg,...,76,75,76,64,74,81,76,73,72,
7,https://cdn.sofifa.org/players/10/19/205670.png,D. Gayle,$9.3M,Dwight Gayle,"ST,LW",England,27,"Oct 17, 1990",174cm,78kg,...,81,83,76,81,76,72,70,62,71,
8,https://cdn.sofifa.org/players/10/19/190362.png,T. Pukki,$9.3M,Teemu Pukki,"ST,CF",Finland,28,"Mar 29, 1990",180cm,80kg,...,78,82,77,72,74,75,79,69,64,
9,https://cdn.sofifa.org/players/10/19/184716.png,J. Allen,$8.7M,Joe Allen,CM,Wales,28,"Mar 14, 1990",168cm,62kg,...,68,82,81,91,67,81,91,60,58,


In [34]:
#GUARDAR DB
da2_England.to_csv('Proyecto/2da_England.csv',index=False,encoding='UTF-16')

In [ ]:
#GUARDAR Y DESCARGAR EN GOOGLE COLAB

#from google.colab import files 

#database.to_csv('db_sofifa.csv',index=False,encoding='UTF-8')
#files.download('db_sofifa.csv')